In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("data_100000.csv")


In [4]:
#No duplicates
df.duplicated().any()

False

In [8]:
print(df.isnull().sum().sum())

34945


In [9]:
#Based on the documentation, if the status is Dead or Stump, information for the columns from health to problems where not recorded
# I changed this NaN values to a category called "Dead_Stump" 
df.loc[(df['status']=='Dead') | (df['status']=='Stump'), 'health'] ="Dead_Stump"

In [10]:
df.loc[(df['health']=='Dead_Stump'), ['health', 'spc_latin', 'spc_common',
       'steward', 'guards', 'sidewalk', 'problems']]='Dead_Stump'

In [11]:
#Values with Yes or No, are changed to Yes or NaN, why? afterward I will conbine all root problems to one column, same with trunks and banches. 
root_problem=['root_stone', 'root_grate', 'root_other']
trunk_problem=['trnk_wire', 'trnk_light', 'trnk_other']
branch_problem=['brnch_ligh', 'brnch_shoe', 'brnch_othe']
all_binaries= root_problem+trunk_problem+branch_problem

In [12]:
for i in all_binaries:
    df.loc[df[i]=="No", i]=np.NaN

In [13]:
df['root_problem']=df[root_problem].any(axis=1)
df['trunk_problem']=df[root_problem].any(axis=1)
df['branch_problem']=df[root_problem].any(axis=1)

In [14]:
df['root_problem']=df['root_problem'].apply(lambda x: 1 if x else 0)
df['trunk_problem']=df['trunk_problem'].apply(lambda x: 1 if x else 0)
df['branch_problem']=df['branch_problem'].apply(lambda x: 1 if x else 0)

In [15]:
#Droping "unnecesary columns"
#all_binaries contains columsn that were prevously cobindine
#columns state is redundant, we know that the dataset is from New York

df.drop(columns=all_binaries, inplace=True)
df.drop(columns=['state'], inplace=True)

In [16]:
#1 NaN value, wich is it?
print(df.isnull().sum().sum())

1


In [24]:
row_without_health= df[df.health.isnull()]
row_without_health

,created_at,tree_id,block_id,the_geom,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,...,nta,nta_name,boro_ct,latitude,longitude,x_sp,y_sp,root_problem,trunk_problem,branch_problem
32794,09/21/2015,245041,413012,POINT (-74.21641202587655 40.54859739520807),16,0,OnCurb,Alive,NaN,Fraxinus pennsylvanica,...,SI32,Rossville-Woodrow,5020801,40.548597,-74.216412,924106.880773,139219.632044,0,0,0


In [25]:
#Are this points duplicated, can there be more than 1 tree at each point? 
count_points= df.the_geom.value_counts()
count_points[count_points>1]

POINT (-73.95680061508556 40.61596593024687)     17
POINT (-73.78450230028402 40.78750376931255)      7
POINT (-73.94127917225204 40.838929366037085)     2
POINT (-73.9533161794278 40.738109223463006)      2
POINT (-73.94074080596141 40.839665395447945)     2
POINT (-73.91800322396026 40.74690571075085)      2
POINT (-73.89274872743376 40.75934013151726)      2
POINT (-73.96331506361766 40.808811547267055)     2
POINT (-73.89603086184627 40.74945433700686)      2
POINT (-73.95281428307973 40.76652028002983)      2
POINT (-73.9751712010072 40.79002417552816)       2
POINT (-73.94156210703247 40.83854197509979)      2
POINT (-73.94377478585902 40.83551862761059)      2
POINT (-73.96260946824727 40.80982868690634)      2
POINT (-73.97798643549005 40.78659233646422)      2
POINT (-73.91832729700161 40.653959833861904)     2
POINT (-73.91628659044508 40.74927713613266)      2
POINT (-73.91780132968749 40.75645762364357)      2
Name: the_geom, dtype: int64

In [12]:
df.to_csv("out.csv")